In [1]:
## La couche d'encodage est le label

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
NB_LABEL = 10

In [5]:
def weight_variable(shape):
    stddev = np.sqrt(1. / np.prod(shape[:-1]))
    print(stddev)
    initial = tf.random_normal(shape, stddev = stddev)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0., shape=shape)
    return tf.Variable(initial)

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [7]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, NB_LABEL])

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

0.2


In [9]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [10]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

0.0353553390593


In [12]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

0.0178571428571


In [13]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_logit = tf.matmul(h_fc1, W_fc2) + b_fc2

0.03125


In [14]:
indices = tf.argmax(y_logit, 1)

# multinomial softmax draw
#indices = tf.multinomial(y_logit, 1)[:,0]

depth = NB_LABEL
y_hat = tf.one_hot(indices, depth)

### Decoder

In [15]:
'''W_fc3 = weight_variable([10, 1024])
b_fc3 = bias_variable([1024])

#h_decoder = tf.nn.relu(tf.matmul(tf.nn.softmax(y_logit), W_fc3) + b_fc3)
h_decoder = tf.nn.relu(tf.matmul(y_hat * y_logit, W_fc3) + b_fc3)
#h_decoder = tf.matmul(y_hat, W_fc3) + b_fc3

W_fc4 = weight_variable([1024, 784])
b_fc4 = bias_variable([784])

x_gen = tf.matmul(h_decoder, W_fc4) + b_fc4'''


W_fc3 = weight_variable([10, 784])
b_fc3 = bias_variable([784])
#x_gen = tf.matmul(tf.nn.softmax(y_logit), W_fc3) + b_fc3
x_gen = tf.matmul(y_hat, W_fc3) + b_fc3

#x_gen = tf.matmul(y_hat * y_logit, W_fc3) + b_fc3


0.316227766017


In [16]:
correct_prediction = tf.cast(tf.equal(indices, tf.argmax(y_, 1)), tf.float32)
reward = correct_prediction - (1 - correct_prediction)
#reward_KL_plus = 1 #correct_prediction
#reward_KL_moins = 1 - correct_prediction

In [17]:
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

p = tf.nn.softmax(y_logit)
q = 1. / NB_LABEL

#KL = tf.reduce_sum(p * y_logit,reduction_indices=1)
KL_ref = tf.reduce_sum(p * tf.log(p/q), reduction_indices=1)
KL = tf.reduce_mean(KL_ref) # + reward_KL_moins * (- tf.log(0.1) - KL_ref))
#KL = tf.nn.softmax_cross_entropy_with_logits(labels=y_hat, logits=y_logit) #!! KL(1_\hat{y})
#KL = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_logit)) #!! KL(1_\hat{y})

#cross_entropy_loss = tf.reduce_mean(reward * tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_gen, labels=x), reduction_indices=1))
#reward = 1
cross_entropy_loss = tf.reduce_mean(reward  * tf.nn.softmax_cross_entropy_with_logits(labels=y_hat, logits=y_logit)) 
#cross_entropy_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_gen, labels=x))

reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_gen, labels=x), reduction_indices = 1))
#rewarded_loss = reward * (KL + cross_entropy_loss)

rewarded_loss = KL +  cross_entropy_loss + reconstruction_loss
                              
train_step = tf.train.AdamOptimizer(1e-4).minimize(rewarded_loss)
accuracy = tf.reduce_mean(correct_prediction)



In [18]:
sess = tf.InteractiveSession()

In [19]:
#sess.run(KL, feed_dict={x: batch[0], y_: batch[1]})
#sess.run(reward, feed_dict={x: batch[0], y_: batch[1]})


In [20]:
#sess.run(rewarded_cross_entropy, feed_dict={x: batch[0], y_: batch[1]})

In [21]:
sess.run(tf.global_variables_initializer())

In [22]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    #print(reward.eval(feed_dict={x: batch[0], y_: batch[1]}))
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]}) #, keep_prob: 1.0})
        consistency_loss = np.mean(KL.eval(feed_dict={x: batch[0], y_: batch[1]})) #, keep_prob: 1.0})
        code_loss = np.mean(cross_entropy_loss.eval(feed_dict={x: batch[0], y_: batch[1]})) #, keep_prob: 1.0})
        reco_loss = np.mean(reconstruction_loss.eval(feed_dict={x: batch[0], y_: batch[1]})) #, keep_prob: 1.0})
        print('step %d, training accuracy %g, consistency loss : %g, code_loss %g, reconstr_loss %g' % (i, train_accuracy, consistency_loss, code_loss,  reco_loss))
    train_step.run(feed_dict={x: batch[0], y_: batch[1]}) #, keep_prob: 0.5})



step 0, training accuracy 0.14, consistency loss : 0.0187655, code_loss -1.46694, reconstr_loss 553.018
step 100, training accuracy 0.86, consistency loss : 0.33715, code_loss 0.619071, reconstr_loss 549.864
step 200, training accuracy 0.98, consistency loss : 0.348999, code_loss 0.951596, reconstr_loss 544.55
step 300, training accuracy 0.98, consistency loss : 0.434332, code_loss 0.802675, reconstr_loss 539.839
step 400, training accuracy 0.98, consistency loss : 0.456486, code_loss 0.763334, reconstr_loss 532.1
step 500, training accuracy 0.94, consistency loss : 0.41078, code_loss 0.724965, reconstr_loss 528.127
step 600, training accuracy 0.96, consistency loss : 0.419169, code_loss 0.755063, reconstr_loss 523.735
step 700, training accuracy 0.98, consistency loss : 0.398874, code_loss 0.805686, reconstr_loss 518.131
step 800, training accuracy 0.98, consistency loss : 0.408243, code_loss 0.811472, reconstr_loss 512.224
step 900, training accuracy 0.94, consistency loss : 0.432623

step 8000, training accuracy 1, consistency loss : 0.458819, code_loss 0.760843, reconstr_loss 297.064
step 8100, training accuracy 1, consistency loss : 0.458472, code_loss 0.756359, reconstr_loss 295.449
step 8200, training accuracy 0.98, consistency loss : 0.457381, code_loss 0.711196, reconstr_loss 276.964
step 8300, training accuracy 1, consistency loss : 0.469771, code_loss 0.747119, reconstr_loss 282.967
step 8400, training accuracy 1, consistency loss : 0.4375, code_loss 0.782222, reconstr_loss 273.692
step 8500, training accuracy 1, consistency loss : 0.467055, code_loss 0.755592, reconstr_loss 267.828
step 8600, training accuracy 1, consistency loss : 0.478861, code_loss 0.731618, reconstr_loss 276
step 8700, training accuracy 1, consistency loss : 0.441532, code_loss 0.775604, reconstr_loss 273.202
step 8800, training accuracy 1, consistency loss : 0.477175, code_loss 0.734638, reconstr_loss 277.613
step 8900, training accuracy 1, consistency loss : 0.453935, code_loss 0.757

step 15900, training accuracy 1, consistency loss : 0.44943, code_loss 0.758739, reconstr_loss 200.468
step 16000, training accuracy 1, consistency loss : 0.465375, code_loss 0.741696, reconstr_loss 207.848
step 16100, training accuracy 1, consistency loss : 0.44967, code_loss 0.759628, reconstr_loss 212.093
step 16200, training accuracy 1, consistency loss : 0.45008, code_loss 0.755776, reconstr_loss 199.311
step 16300, training accuracy 0.98, consistency loss : 0.433175, code_loss 0.712516, reconstr_loss 196.906
step 16400, training accuracy 1, consistency loss : 0.461725, code_loss 0.746197, reconstr_loss 206.833
step 16500, training accuracy 1, consistency loss : 0.488065, code_loss 0.719333, reconstr_loss 196.669
step 16600, training accuracy 1, consistency loss : 0.474694, code_loss 0.730181, reconstr_loss 207.869
step 16700, training accuracy 1, consistency loss : 0.462231, code_loss 0.743948, reconstr_loss 209.492
step 16800, training accuracy 1, consistency loss : 0.467875, co

In [ ]:
#cross_entropy_loss.eval(feed_dict={x: batch[0], y_: batch[1]})
indices.eval(feed_dict={x: batch[0], y_: batch[1]})

array([9, 8, 0, 6, 6, 1, 1, 8, 8, 6, 8, 9, 7, 2, 1, 1, 7, 3, 2, 7, 1, 2, 4,
       2, 0, 9, 6, 6, 1, 8, 1, 7, 6, 9, 1, 7, 9, 6, 2, 1, 5, 0, 8, 1, 0, 0,
       7, 2, 1, 1])

In [ ]:
print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
code = sess.run(tf.nn.softmax(y_logit), feed_dict={x: batch[0], y_: batch[1]})
pred = sess.run(x_gen, feed_dict={x: batch[0], y_: batch[1]})
i = 0
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(np.reshape(batch[0][i],(28, 28)))
plt.subplot(1,3,2)
plt.plot(code[i])
plt.subplot(1,3,3)
plt.imshow(np.reshape(pred[i],(28, 28)))



In [ ]:
reward

In [ ]:
sess.run((tf.multinomial(y_logit, 1)[:,0], indices), feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
-np.log(0.1)